<a href="https://colab.research.google.com/github/bwbayu/TalentMatch/blob/development/model/training/training_SBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/data_supervised.csv')
df.head(5)

,Unnamed: 0,clean_cv,clean_jd,label
0,0,result oriented organized bilingual accounting...,minimum education requirement bachelor degree ...,1
1,1,result oriented organized bilingual accounting...,hiring talented candidate join accounting team...,1
2,2,result oriented organized bilingual accounting...,duty proficient working excel skilled working ...,1
3,3,result oriented organized bilingual accounting...,job description responsible supervising checki...,1
4,4,result oriented organized bilingual accounting...,qualification one bachelor degree finance acco...,1


In [ ]:
df = df[['clean_cv', 'clean_jd', 'label']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31203 entries, 0 to 31202
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   clean_cv  31203 non-null  object
 1   clean_jd  31203 non-null  object
 2   label     31203 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 731.4+ KB


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df['label'].value_counts()

label
1    12552
0    12410
Name: count, dtype: int64

In [ ]:
test_df['label'].value_counts()

label
1    3166
0    3075
Name: count, dtype: int64

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import CoSENTLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# model = SentenceTransformer(
#     "msmarco-distilbert-base-tas-b",
# )

model = SentenceTransformer(
    "/content/drive/MyDrive/model/sbert/model_30k_4_5e6",
)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

train_columns = ['clean_cv', 'clean_jd', 'label']
test_columns = ['clean_cv', 'clean_jd', 'label']

train_dataset = train_dataset.select_columns(train_columns)
eval_dataset = eval_dataset.select_columns(test_columns)

loss = CoSENTLoss(model)

# 4. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/mpnet-base-custom-dataset",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-6,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="mpnet-base-custom-dataset",  # Will be used in W&B if `wandb` is installed
)

# 5. (Optional) Create an evaluator & evaluate the base model
# Define an evaluation metric if needed, e.g., Accuracy, Precision, etc.
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["clean_cv"],
    sentences2=eval_dataset["clean_jd"],
    scores=eval_dataset["label"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

dev_evaluator(model)

# 6. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
100,0.238900,0.502786,0.926673,0.857840,0.914688,0.859718,0.916441,0.859957,0.873714,0.850244,0.926673,0.859957
200,0.219600,0.456877,0.927912,0.858415,0.911209,0.860014,0.912114,0.859791,0.877433,0.851157,0.927912,0.860014
300,0.197100,0.711404,0.918055,0.853568,0.911155,0.856680,0.913424,0.857775,0.867299,0.846529,0.918055,0.857775
400,0.196800,0.574667,0.921828,0.855871,0.912426,0.858238,0.913969,0.858704,0.871127,0.849400,0.921828,0.858704
500,0.161800,0.561577,0.924519,0.856290,0.910056,0.857071,0.911536,0.856976,0.874925,0.849574,0.924519,0.857071
600,0.184800,0.500988,0.924824,0.857585,0.915125,0.857204,0.916643,0.857531,0.876118,0.851400,0.924824,0.857585
700,0.079700,0.739682,0.918463,0.853302,0.908045,0.854934,0.910043,0.855594,0.868411,0.846486,0.918463,0.855594
800,0.154000,0.775793,0.910125,0.852067,0.905031,0.853932,0.908526,0.855884,0.858277,0.845997,0.910125,0.855884
900,0.132400,0.435065,0.929978,0.859224,0.920423,0.860321,0.922230,0.860271,0.878361,0.853042,0.929978,0.860321
1000,0.128400,0.664511,0.917300,0.854127,0.909550,0.854619,0.913907,0.856622,0.873785,0.849535,0.917300,0.856622


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model.save_pretrained("/content/drive/MyDrive/model/sbert/model_30k_7_5e6")